In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import train_test_split,GridSearchCV

In [67]:
df= pd.read_csv('horse.csv')

In [68]:
df.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,outcome,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,no,adult,530101,38.5,66.0,28.0,cool,reduced,NaN,more_3_sec,...,45.0,8.4,NaN,NaN,died,no,11300,0,0,no
1,yes,adult,534817,39.2,88.0,20.0,NaN,NaN,pale_cyanotic,less_3_sec,...,50.0,85.0,cloudy,2.0,euthanized,no,2208,0,0,no
2,no,adult,530334,38.3,40.0,24.0,normal,normal,pale_pink,less_3_sec,...,33.0,6.7,NaN,NaN,lived,no,0,0,0,yes
3,yes,young,5290409,39.1,164.0,84.0,cold,normal,dark_cyanotic,more_3_sec,...,48.0,7.2,serosanguious,5.3,died,yes,2208,0,0,yes
4,no,adult,530255,37.3,104.0,35.0,NaN,NaN,dark_cyanotic,more_3_sec,...,74.0,7.4,NaN,NaN,died,no,4300,0,0,no


In [69]:
df.shape

(299, 28)

In [70]:
target=df[['outcome']]

In [71]:
features= df.drop(columns='outcome')

In [72]:
features.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,no,adult,530101,38.5,66.0,28.0,cool,reduced,NaN,more_3_sec,...,distend_large,45.0,8.4,NaN,NaN,no,11300,0,0,no
1,yes,adult,534817,39.2,88.0,20.0,NaN,NaN,pale_cyanotic,less_3_sec,...,other,50.0,85.0,cloudy,2.0,no,2208,0,0,no
2,no,adult,530334,38.3,40.0,24.0,normal,normal,pale_pink,less_3_sec,...,normal,33.0,6.7,NaN,NaN,no,0,0,0,yes
3,yes,young,5290409,39.1,164.0,84.0,cold,normal,dark_cyanotic,more_3_sec,...,NaN,48.0,7.2,serosanguious,5.3,yes,2208,0,0,yes
4,no,adult,530255,37.3,104.0,35.0,NaN,NaN,dark_cyanotic,more_3_sec,...,NaN,74.0,7.4,NaN,NaN,no,4300,0,0,no


In [73]:
features.isnull().sum()

surgery                    0
age                        0
hospital_number            0
rectal_temp               60
pulse                     24
respiratory_rate          58
temp_of_extremities       56
peripheral_pulse          69
mucous_membrane           47
capillary_refill_time     32
pain                      55
peristalsis               44
abdominal_distention      56
nasogastric_tube         104
nasogastric_reflux       106
nasogastric_reflux_ph    246
rectal_exam_feces        102
abdomen                  118
packed_cell_volume        29
total_protein             33
abdomo_appearance        165
abdomo_protein           198
surgical_lesion            0
lesion_1                   0
lesion_2                   0
lesion_3                   0
cp_data                    0
dtype: int64

In [74]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   surgery                299 non-null    object 
 1   age                    299 non-null    object 
 2   hospital_number        299 non-null    int64  
 3   rectal_temp            239 non-null    float64
 4   pulse                  275 non-null    float64
 5   respiratory_rate       241 non-null    float64
 6   temp_of_extremities    243 non-null    object 
 7   peripheral_pulse       230 non-null    object 
 8   mucous_membrane        252 non-null    object 
 9   capillary_refill_time  267 non-null    object 
 10  pain                   244 non-null    object 
 11  peristalsis            255 non-null    object 
 12  abdominal_distention   243 non-null    object 
 13  nasogastric_tube       195 non-null    object 
 14  nasogastric_reflux     193 non-null    object 
 15  nasoga

In [75]:
features.columns

Index(['surgery', 'age', 'hospital_number', 'rectal_temp', 'pulse',
       'respiratory_rate', 'temp_of_extremities', 'peripheral_pulse',
       'mucous_membrane', 'capillary_refill_time', 'pain', 'peristalsis',
       'abdominal_distention', 'nasogastric_tube', 'nasogastric_reflux',
       'nasogastric_reflux_ph', 'rectal_exam_feces', 'abdomen',
       'packed_cell_volume', 'total_protein', 'abdomo_appearance',
       'abdomo_protein', 'surgical_lesion', 'lesion_1', 'lesion_2', 'lesion_3',
       'cp_data'],
      dtype='object')

In [76]:
cat_col= [i for i in features.columns if features[i].dtype=='O']
print(cat_col)

['surgery', 'age', 'temp_of_extremities', 'peripheral_pulse', 'mucous_membrane', 'capillary_refill_time', 'pain', 'peristalsis', 'abdominal_distention', 'nasogastric_tube', 'nasogastric_reflux', 'rectal_exam_feces', 'abdomen', 'abdomo_appearance', 'surgical_lesion', 'cp_data']


In [77]:
num_col= [i for i in features.columns if features[i].dtype!='O']
print(num_col)

['hospital_number', 'rectal_temp', 'pulse', 'respiratory_rate', 'nasogastric_reflux_ph', 'packed_cell_volume', 'total_protein', 'abdomo_protein', 'lesion_1', 'lesion_2', 'lesion_3']


In [78]:
features[cat_col].head()

,surgery,age,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,rectal_exam_feces,abdomen,abdomo_appearance,surgical_lesion,cp_data
0,no,adult,cool,reduced,NaN,more_3_sec,extreme_pain,absent,severe,NaN,NaN,decreased,distend_large,NaN,no,no
1,yes,adult,NaN,NaN,pale_cyanotic,less_3_sec,mild_pain,absent,slight,NaN,NaN,absent,other,cloudy,no,no
2,no,adult,normal,normal,pale_pink,less_3_sec,mild_pain,hypomotile,none,NaN,NaN,normal,normal,NaN,no,yes
3,yes,young,cold,normal,dark_cyanotic,more_3_sec,depressed,absent,severe,none,less_1_liter,decreased,NaN,serosanguious,yes,yes
4,no,adult,NaN,NaN,dark_cyanotic,more_3_sec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,no


In [79]:
features[num_col].head()

,hospital_number,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,lesion_3
0,530101,38.5,66.0,28.0,NaN,45.0,8.4,NaN,11300,0,0
1,534817,39.2,88.0,20.0,NaN,50.0,85.0,2.0,2208,0,0
2,530334,38.3,40.0,24.0,NaN,33.0,6.7,NaN,0,0,0
3,5290409,39.1,164.0,84.0,5.0,48.0,7.2,5.3,2208,0,0
4,530255,37.3,104.0,35.0,NaN,74.0,7.4,NaN,4300,0,0


In [80]:
## cat_col
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

In [81]:
ord_enc= make_pipeline(OrdinalEncoder())
simp_imp= SimpleImputer(missing_values=np.nan,strategy='most_frequent')

In [82]:
transformer=ColumnTransformer([('ordinal',ord_enc,cat_col)])

In [83]:
transformer.get_feature_names_out

<bound method ColumnTransformer.get_feature_names_out of ColumnTransformer(transformers=[('ordinal',
                                 Pipeline(steps=[('ordinalencoder',
                                                  OrdinalEncoder())]),
                                 ['surgery', 'age', 'temp_of_extremities',
                                  'peripheral_pulse', 'mucous_membrane',
                                  'capillary_refill_time', 'pain',
                                  'peristalsis', 'abdominal_distention',
                                  'nasogastric_tube', 'nasogastric_reflux',
                                  'rectal_exam_feces', 'abdomen',
                                  'abdomo_appearance', 'surgical_lesion',
                                  'cp_data'])])>

In [84]:
df1=pd.DataFrame(transformer.fit_transform(features[cat_col]),columns=transformer.get_feature_names_out())

In [85]:
df1=pd.DataFrame(simp_imp.fit_transform(df1),columns=df1.columns)

In [86]:
df1

,ordinal__surgery,ordinal__age,ordinal__temp_of_extremities,ordinal__peripheral_pulse,ordinal__mucous_membrane,ordinal__capillary_refill_time,ordinal__pain,ordinal__peristalsis,ordinal__abdominal_distention,ordinal__nasogastric_tube,ordinal__nasogastric_reflux,ordinal__rectal_exam_feces,ordinal__abdomen,ordinal__abdomo_appearance,ordinal__surgical_lesion,ordinal__cp_data
0,0.0,0.0,1.0,3.0,3.0,2.0,2.0,0.0,2.0,2.0,2.0,1.0,0.0,1.0,0.0,0.0
1,1.0,0.0,1.0,2.0,4.0,1.0,3.0,0.0,3.0,2.0,2.0,0.0,4.0,1.0,0.0,0.0
2,0.0,0.0,2.0,2.0,5.0,1.0,3.0,2.0,1.0,2.0,2.0,3.0,3.0,1.0,0.0,1.0
3,1.0,1.0,0.0,2.0,2.0,2.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,1.0,1.0
4,0.0,0.0,1.0,2.0,2.0,2.0,3.0,2.0,1.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,1.0,0.0,0.0,2.0,4.0,2.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0
295,0.0,0.0,1.0,1.0,4.0,2.0,4.0,2.0,0.0,1.0,2.0,0.0,1.0,2.0,1.0,1.0
296,1.0,0.0,0.0,3.0,4.0,1.0,4.0,0.0,0.0,2.0,2.0,1.0,0.0,1.0,1.0,0.0
297,1.0,0.0,1.0,3.0,5.0,1.0,3.0,2.0,0.0,1.0,2.0,0.0,1.0,2.0,1.0,1.0


In [87]:
## num_col
simp_imp1= SimpleImputer(missing_values=np.nan,strategy='median')
df2=pd.DataFrame(simp_imp1.fit_transform(features[num_col]),columns=num_col)

In [88]:
from sklearn.preprocessing import StandardScaler
std_sca= StandardScaler()

In [89]:
df2=pd.DataFrame(std_sca.fit_transform(df2),columns=df2.columns)

In [90]:
df2.head()

,hospital_number,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,lesion_3
0,-0.364592,0.496498,-0.194771,-0.087726,0.062143,-0.119170,-0.533355,-0.209831,1.415021,-0.139371,-0.057928
1,-0.361509,1.565603,0.604982,-0.588538,0.062143,0.385530,2.380057,-0.461797,-0.268864,-0.139371,-0.057928
2,-0.364440,0.191039,-1.139933,-0.338132,0.062143,-1.330449,-0.598013,-0.209831,-0.677797,-0.139371,-0.057928
3,2.747804,1.412873,3.367763,3.417962,0.062143,0.183650,-0.578996,2.309828,-0.268864,-0.139371,-0.057928
4,-0.364491,-1.336253,1.186620,0.350485,0.062143,2.808088,-0.571389,-0.209831,0.118585,-0.139371,-0.057928


In [91]:
df_new=pd.concat([df1,df2],axis=1)

In [92]:
df_new

,ordinal__surgery,ordinal__age,ordinal__temp_of_extremities,ordinal__peripheral_pulse,ordinal__mucous_membrane,ordinal__capillary_refill_time,ordinal__pain,ordinal__peristalsis,ordinal__abdominal_distention,ordinal__nasogastric_tube,...,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,lesion_3
0,0.0,0.0,1.0,3.0,3.0,2.0,2.0,0.0,2.0,2.0,...,0.496498,-0.194771,-0.087726,0.062143,-0.119170,-0.533355,-0.209831,1.415021,-0.139371,-0.057928
1,1.0,0.0,1.0,2.0,4.0,1.0,3.0,0.0,3.0,2.0,...,1.565603,0.604982,-0.588538,0.062143,0.385530,2.380057,-0.461797,-0.268864,-0.139371,-0.057928
2,0.0,0.0,2.0,2.0,5.0,1.0,3.0,2.0,1.0,2.0,...,0.191039,-1.139933,-0.338132,0.062143,-1.330449,-0.598013,-0.209831,-0.677797,-0.139371,-0.057928
3,1.0,1.0,0.0,2.0,2.0,2.0,1.0,0.0,2.0,0.0,...,1.412873,3.367763,3.417962,0.062143,0.183650,-0.578996,2.309828,-0.268864,-0.139371,-0.057928
4,0.0,0.0,1.0,2.0,2.0,2.0,3.0,2.0,1.0,2.0,...,-1.336253,1.186620,0.350485,0.062143,2.808088,-0.571389,-0.209831,0.118585,-0.139371,-0.057928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,1.0,0.0,0.0,2.0,4.0,2.0,1.0,0.0,1.0,2.0,...,0.038310,1.768258,2.541540,0.062143,0.890229,1.619375,-0.209831,-0.084214,-0.139371,-0.057928
295,0.0,0.0,1.0,1.0,4.0,2.0,4.0,2.0,0.0,1.0,...,-1.488983,0.023343,-0.338132,0.062143,-0.220110,-0.567586,0.630055,-0.268864,-0.139371,-0.057928
296,1.0,0.0,0.0,3.0,4.0,1.0,4.0,0.0,0.0,2.0,...,-1.030795,0.023343,0.037477,0.062143,1.394929,-0.594210,-0.209831,-0.084214,-0.139371,-0.057928
297,1.0,0.0,1.0,3.0,5.0,1.0,3.0,2.0,0.0,1.0,...,-2.558088,1.041210,-0.338132,0.062143,0.385530,-0.624637,0.714044,-0.268864,-0.139371,-0.057928


In [93]:
import scipy.stats as stats

In [94]:
## outlier removal
def outlier_remove_z_score(df,columns):
    thershold= 3
    z_score= stats.zscore(df[columns])
    return df[(abs(z_score)<thershold).all(axis=1)]

In [95]:
df_new1=outlier_remove_z_score(df_new,df_new.columns)

In [96]:
df_new1

,ordinal__surgery,ordinal__age,ordinal__temp_of_extremities,ordinal__peripheral_pulse,ordinal__mucous_membrane,ordinal__capillary_refill_time,ordinal__pain,ordinal__peristalsis,ordinal__abdominal_distention,ordinal__nasogastric_tube,...,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,lesion_3
0,0.0,0.0,1.0,3.0,3.0,2.0,2.0,0.0,2.0,2.0,...,0.496498,-0.194771,-0.087726,0.062143,-0.119170,-0.533355,-0.209831,1.415021,-0.139371,-0.057928
1,1.0,0.0,1.0,2.0,4.0,1.0,3.0,0.0,3.0,2.0,...,1.565603,0.604982,-0.588538,0.062143,0.385530,2.380057,-0.461797,-0.268864,-0.139371,-0.057928
2,0.0,0.0,2.0,2.0,5.0,1.0,3.0,2.0,1.0,2.0,...,0.191039,-1.139933,-0.338132,0.062143,-1.330449,-0.598013,-0.209831,-0.677797,-0.139371,-0.057928
4,0.0,0.0,1.0,2.0,2.0,2.0,3.0,2.0,1.0,2.0,...,-1.336253,1.186620,0.350485,0.062143,2.808088,-0.571389,-0.209831,0.118585,-0.139371,-0.057928
5,0.0,0.0,3.0,2.0,5.0,1.0,1.0,2.0,3.0,2.0,...,0.038310,-0.267476,-0.275531,0.062143,-0.119170,-0.567586,-0.209831,-0.677797,-0.139371,-0.057928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,1.0,0.0,0.0,2.0,4.0,2.0,1.0,0.0,1.0,2.0,...,0.038310,1.768258,2.541540,0.062143,0.890229,1.619375,-0.209831,-0.084214,-0.139371,-0.057928
295,0.0,0.0,1.0,1.0,4.0,2.0,4.0,2.0,0.0,1.0,...,-1.488983,0.023343,-0.338132,0.062143,-0.220110,-0.567586,0.630055,-0.268864,-0.139371,-0.057928
296,1.0,0.0,0.0,3.0,4.0,1.0,4.0,0.0,0.0,2.0,...,-1.030795,0.023343,0.037477,0.062143,1.394929,-0.594210,-0.209831,-0.084214,-0.139371,-0.057928
297,1.0,0.0,1.0,3.0,5.0,1.0,3.0,2.0,0.0,1.0,...,-2.558088,1.041210,-0.338132,0.062143,0.385530,-0.624637,0.714044,-0.268864,-0.139371,-0.057928


In [97]:
df_new1

,ordinal__surgery,ordinal__age,ordinal__temp_of_extremities,ordinal__peripheral_pulse,ordinal__mucous_membrane,ordinal__capillary_refill_time,ordinal__pain,ordinal__peristalsis,ordinal__abdominal_distention,ordinal__nasogastric_tube,...,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,lesion_3
0,0.0,0.0,1.0,3.0,3.0,2.0,2.0,0.0,2.0,2.0,...,0.496498,-0.194771,-0.087726,0.062143,-0.119170,-0.533355,-0.209831,1.415021,-0.139371,-0.057928
1,1.0,0.0,1.0,2.0,4.0,1.0,3.0,0.0,3.0,2.0,...,1.565603,0.604982,-0.588538,0.062143,0.385530,2.380057,-0.461797,-0.268864,-0.139371,-0.057928
2,0.0,0.0,2.0,2.0,5.0,1.0,3.0,2.0,1.0,2.0,...,0.191039,-1.139933,-0.338132,0.062143,-1.330449,-0.598013,-0.209831,-0.677797,-0.139371,-0.057928
4,0.0,0.0,1.0,2.0,2.0,2.0,3.0,2.0,1.0,2.0,...,-1.336253,1.186620,0.350485,0.062143,2.808088,-0.571389,-0.209831,0.118585,-0.139371,-0.057928
5,0.0,0.0,3.0,2.0,5.0,1.0,1.0,2.0,3.0,2.0,...,0.038310,-0.267476,-0.275531,0.062143,-0.119170,-0.567586,-0.209831,-0.677797,-0.139371,-0.057928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,1.0,0.0,0.0,2.0,4.0,2.0,1.0,0.0,1.0,2.0,...,0.038310,1.768258,2.541540,0.062143,0.890229,1.619375,-0.209831,-0.084214,-0.139371,-0.057928
295,0.0,0.0,1.0,1.0,4.0,2.0,4.0,2.0,0.0,1.0,...,-1.488983,0.023343,-0.338132,0.062143,-0.220110,-0.567586,0.630055,-0.268864,-0.139371,-0.057928
296,1.0,0.0,0.0,3.0,4.0,1.0,4.0,0.0,0.0,2.0,...,-1.030795,0.023343,0.037477,0.062143,1.394929,-0.594210,-0.209831,-0.084214,-0.139371,-0.057928
297,1.0,0.0,1.0,3.0,5.0,1.0,3.0,2.0,0.0,1.0,...,-2.558088,1.041210,-0.338132,0.062143,0.385530,-0.624637,0.714044,-0.268864,-0.139371,-0.057928


In [98]:
target.shape

(299, 1)

In [99]:
df_n=pd.merge(df_new1,target,left_index=True,right_index=True)

In [100]:
df_n

,ordinal__surgery,ordinal__age,ordinal__temp_of_extremities,ordinal__peripheral_pulse,ordinal__mucous_membrane,ordinal__capillary_refill_time,ordinal__pain,ordinal__peristalsis,ordinal__abdominal_distention,ordinal__nasogastric_tube,...,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,lesion_3,outcome
0,0.0,0.0,1.0,3.0,3.0,2.0,2.0,0.0,2.0,2.0,...,-0.194771,-0.087726,0.062143,-0.119170,-0.533355,-0.209831,1.415021,-0.139371,-0.057928,died
1,1.0,0.0,1.0,2.0,4.0,1.0,3.0,0.0,3.0,2.0,...,0.604982,-0.588538,0.062143,0.385530,2.380057,-0.461797,-0.268864,-0.139371,-0.057928,euthanized
2,0.0,0.0,2.0,2.0,5.0,1.0,3.0,2.0,1.0,2.0,...,-1.139933,-0.338132,0.062143,-1.330449,-0.598013,-0.209831,-0.677797,-0.139371,-0.057928,lived
4,0.0,0.0,1.0,2.0,2.0,2.0,3.0,2.0,1.0,2.0,...,1.186620,0.350485,0.062143,2.808088,-0.571389,-0.209831,0.118585,-0.139371,-0.057928,died
5,0.0,0.0,3.0,2.0,5.0,1.0,1.0,2.0,3.0,2.0,...,-0.267476,-0.275531,0.062143,-0.119170,-0.567586,-0.209831,-0.677797,-0.139371,-0.057928,lived
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,1.0,0.0,0.0,2.0,4.0,2.0,1.0,0.0,1.0,2.0,...,1.768258,2.541540,0.062143,0.890229,1.619375,-0.209831,-0.084214,-0.139371,-0.057928,euthanized
295,0.0,0.0,1.0,1.0,4.0,2.0,4.0,2.0,0.0,1.0,...,0.023343,-0.338132,0.062143,-0.220110,-0.567586,0.630055,-0.268864,-0.139371,-0.057928,euthanized
296,1.0,0.0,0.0,3.0,4.0,1.0,4.0,0.0,0.0,2.0,...,0.023343,0.037477,0.062143,1.394929,-0.594210,-0.209831,-0.084214,-0.139371,-0.057928,died
297,1.0,0.0,1.0,3.0,5.0,1.0,3.0,2.0,0.0,1.0,...,1.041210,-0.338132,0.062143,0.385530,-0.624637,0.714044,-0.268864,-0.139371,-0.057928,lived


In [101]:
target= df_n[['outcome']]
features= df_n.drop(columns='outcome')

In [102]:
x_train,x_test,y_train,y_test= train_test_split(features,target,train_size=0.75,random_state=100)

In [103]:
x_train

,ordinal__surgery,ordinal__age,ordinal__temp_of_extremities,ordinal__peripheral_pulse,ordinal__mucous_membrane,ordinal__capillary_refill_time,ordinal__pain,ordinal__peristalsis,ordinal__abdominal_distention,ordinal__nasogastric_tube,...,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,lesion_3
267,1.0,0.0,1.0,2.0,3.0,1.0,3.0,2.0,1.0,2.0,...,0.038310,-1.067228,-0.588538,0.062143,0.082710,1.429205,-0.209831,-0.677797,-0.139371,-0.057928
196,0.0,0.0,1.0,3.0,3.0,2.0,1.0,0.0,3.0,0.0,...,0.496498,0.895801,0.413087,0.062143,2.404328,-0.529552,-0.209831,-0.418510,-0.139371,-0.057928
132,0.0,0.0,2.0,2.0,3.0,1.0,1.0,2.0,1.0,2.0,...,-0.419878,-1.139933,-0.338132,0.062143,-0.623870,-0.636047,-0.209831,-0.603715,-0.139371,-0.057928
45,0.0,0.0,0.0,3.0,4.0,2.0,2.0,0.0,2.0,0.0,...,0.038310,1.768258,-0.275531,0.062143,1.394929,-0.605620,-0.209831,-0.084214,-0.139371,-0.057928
38,0.0,0.0,2.0,2.0,3.0,1.0,3.0,2.0,3.0,0.0,...,0.649227,-0.703704,-0.275531,0.062143,-1.431389,-0.601817,2.057862,-0.677797,-0.139371,-0.057928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,1.0,0.0,2.0,2.0,0.0,1.0,1.0,2.0,3.0,0.0,...,1.871061,-0.631000,2.291133,0.062143,-0.724809,-0.624637,-0.209831,-0.269234,-0.139371,-0.057928
139,1.0,0.0,1.0,2.0,3.0,1.0,1.0,3.0,1.0,0.0,...,-0.114419,-0.994524,-1.277156,0.062143,-1.532329,-0.575193,-0.209831,0.271194,-0.139371,-0.057928
88,0.0,0.0,2.0,2.0,1.0,1.0,3.0,2.0,3.0,2.0,...,1.260144,-0.267476,0.663493,0.062143,-0.421990,-0.567586,-0.209831,0.248229,-0.139371,-0.057928
32,1.0,0.0,3.0,2.0,3.0,1.0,3.0,2.0,0.0,2.0,...,-1.488983,-0.412885,-0.275531,0.062143,-0.321050,-0.601817,-0.209831,-0.268679,-0.139371,-0.057928


## Bagging_Classifier

In [104]:
x_train.shape

(167, 27)

In [105]:
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression

In [106]:
bag_class= BaggingClassifier(estimator=LogisticRegression(),n_estimators=100,bootstrap=True,max_features=0.9,random_state=50,oob_score=True)

In [107]:
bag_class.fit(x_train,y_train)
import warnings
warnings.filterwarnings('ignore')

In [108]:
bag_class.oob_score_

0.7005988023952096

In [109]:
y_pred= bag_class.predict(x_test)

In [110]:
accuracy_score(y_test,y_pred)

0.7321428571428571

In [111]:
np.log2(27)

4.754887502163468

## Random_Forest

In [112]:
from sklearn.ensemble import RandomForestClassifier

In [113]:
rand_forest= RandomForestClassifier(n_estimators=40,bootstrap=True,max_depth=6,max_features='sqrt',
                                    oob_score=True,criterion='entropy',random_state=100)

In [114]:
rand_forest.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=6, n_estimators=40,
                       oob_score=True, random_state=100)

In [115]:
rand_forest.oob_score_

0.7125748502994012

In [116]:
y_pred= rand_forest.predict(x_test)

In [117]:
accuracy_score(y_test,y_pred)

0.7142857142857143

In [54]:
params= {'n_estimators':[10,30,40,50,100,120,150],'max_depth':[1,2,3,4,5,6],'criterion':['gini','entropy']}

In [55]:
grid_search= GridSearchCV(rand_forest,params)

In [56]:
grid_search.fit(x_train,y_train)

GridSearchCV(estimator=RandomForestClassifier(criterion='entropy', max_depth=6,
                                              n_estimators=40, oob_score=True,
                                              random_state=100),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [1, 2, 3, 4, 5, 6],
                         'n_estimators': [10, 30, 40, 50, 100, 120, 150]})

In [57]:
grid_search.best_params_

{'criterion': 'entropy', 'max_depth': 6, 'n_estimators': 40}

In [58]:
log_odds=np.log(2)
log_odds

0.6931471805599453

In [61]:
np.exp(log_odds)/(1+np.exp(log_odds))

0.6666666666666666

In [62]:
0.3/0.21

1.4285714285714286

In [63]:
np.exp(2.1)/(1+np.exp(2.1))

0.8909031788043871

## Gradient_Boosting

In [118]:
from sklearn.ensemble import GradientBoostingClassifier

In [119]:
grad_boost= GradientBoostingClassifier(n_estimators=100,learning_rate=0.5,max_features=0.9,subsample=0.9)

In [120]:
grad_boost.fit(x_train,y_train)

GradientBoostingClassifier(learning_rate=0.5, max_features=0.9, subsample=0.9)

In [121]:
y_test=grad_boost.predict(x_test)

In [122]:
accuracy_score(y_test,y_pred)

0.8214285714285714

## XG BOOST

In [123]:
!pip install xgboost

In [126]:
y_test

array(['died', 'died', 'lived', 'died', 'died', 'lived', 'lived', 'lived',
       'euthanized', 'died', 'died', 'lived', 'lived', 'euthanized',
       'lived', 'lived', 'died', 'lived', 'lived', 'lived', 'lived',
       'lived', 'lived', 'died', 'euthanized', 'lived', 'died', 'died',
       'lived', 'lived', 'lived', 'lived', 'died', 'lived', 'lived',
       'died', 'died', 'died', 'lived', 'died', 'lived', 'lived', 'lived',
       'died', 'lived', 'died', 'lived', 'lived', 'died', 'lived',
       'lived', 'lived', 'lived', 'lived', 'lived', 'died'], dtype=object)

In [131]:
import xgboost as xgb

In [132]:
xgb= xgb.XGBClassifier(n_estimators=100,min_child_weight=2, learning_rate=0.5,reg_lambda=1,reg_alpha=2)

In [133]:
from sklearn.preprocessing import LabelEncoder
lab_enc= LabelEncoder()
y_train= lab_enc.fit_transform(y_train)
y_test= lab_enc.fit_transform(y_test)

In [134]:
y_test

array([0, 0, 2, 0, 0, 2, 2, 2, 1, 0, 0, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2,
       2, 0, 1, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 0, 0, 0, 2, 0, 2, 2, 2, 0,
       2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0], dtype=int64)

In [135]:
xgb.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.5, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [136]:
y_pred=xgb.predict(x_test)

In [137]:
accuracy_score(y_test,y_pred)

0.9107142857142857

## LightGBM

In [138]:
!pip install lightgbm

In [139]:
import lightgbm as lgb

In [140]:
lgb=lgb.LGBMClassifier(n_estimator=100,min_child_weight=2,learning_rate=0.4,subsample=0.9,max_bins=30,max_depth=3,boosting_type='goss',top_rate=0.3,other_rate=0.2)

In [141]:
lgb.fit(x_train,y_train)

[LightGBM] [Warning] Unknown parameter: n_estimator
[LightGBM] [Warning] Found boosting=goss. For backwards compatibility reasons, LightGBM interprets this as boosting=gbdt, data_sample_strategy=goss.To suppress this warning, set data_sample_strategy=goss instead.
[LightGBM] [Warning] Unknown parameter: n_estimator
[LightGBM] [Warning] Found boosting=goss. For backwards compatibility reasons, LightGBM interprets this as boosting=gbdt, data_sample_strategy=goss.To suppress this warning, set data_sample_strategy=goss instead.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000974 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 167, number of used features: 23
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score -1.454432
[LightGBM] [Info] Start training from

LGBMClassifier(boosting_type='goss', learning_rate=0.4, max_bins=30,
               max_depth=3, min_child_weight=2, n_estimator=100, other_rate=0.2,
               subsample=0.9, top_rate=0.3)

In [142]:
y_pred= lgb.predict(x_test)

[LightGBM] [Warning] Unknown parameter: n_estimator
[LightGBM] [Warning] Found boosting=goss. For backwards compatibility reasons, LightGBM interprets this as boosting=gbdt, data_sample_strategy=goss.To suppress this warning, set data_sample_strategy=goss instead.


In [143]:
accuracy_score(y_test,y_pred)

0.8571428571428571

In [59]:
0.3/0.21

1.4285714285714286

In [60]:
(-0.7-0.7+0.3)/0.62

-1.7741935483870965

In [62]:
np.exp(-1.07)/(1+np.exp(-1.07))

0.255403084172524

In [64]:
np.log(4/3)

0.28768207245178085

In [65]:
np.exp(0.288)/(1+np.exp(0.288))

0.5715064294675559